# Anatomy of ring and motor-related neurons in the hindbrain

!! Require v1.0 of the MPIN atlas!

In [ ]:
%matplotlib widget
import numpy as np
import pandas as pd

import lotr.plotting as pltltr
from lotr import LotrExperiment, dataset_folders

COLS = pltltr.COLS
from matplotlib import pyplot as plt
from tqdm import tqdm

from bg_atlasapi import BrainGlobeAtlas

from lotr.data_preprocessing.anatomy import transform_points
from lotr.em.transformations import ipn_to_mpin
from lotr.utils import circular_corr
from lotr.data_preprocessing.anatomy import anatomical_angle_remapping

## Localization of ring neurons
First, let's have a look at the positions of ring neurons. We will also load information about the motor regressors, as later we will also plot the localizations of the motor-selective neurons.

In [ ]:
Z_SCATTER_FACTOR = 10  # add some random dispersion in z to make all rois visible
data_df = []
for path in tqdm(dataset_folders):
    exp = LotrExperiment(path)
    cent_coords = exp.morphed_coords_um
    data_dict = dict()
    for i, lab in enumerate(["s_i", "l_r", "a_p"]):
        data_dict.update({lab: cent_coords[:, i]})
    data_dict["s_i"] = (
        data_dict["s_i"] + np.random.rand(len(data_dict["a_p"])) * Z_SCATTER_FACTOR
    )
    data_dict["hdn"] = ~exp.nonhdn_indexes

    new_df = pd.concat([pd.DataFrame(data_dict), exp.motor_regressors], axis=1)
    new_df["fid"] = path.name
    data_df.append(new_df)

data_df = pd.concat(data_df, ignore_index=1)

First, a nice horizontal view:

In [ ]:
f, ax = plt.subplots(figsize=(3, 3.5))
s = 8
ax.scatter(
    data_df["l_r"],
    data_df["a_p"],
    lw=0,
    s=s,
    alpha=0.03,
    color=(0.3,) * 3,
    rasterized=True,
)
loc = [-110, -138]
ax.axis("equal")
pltltr.add_anatomy_scalebar(ax, pos=loc, length=40, fontsize=6)

pltltr.savefig("horview_allneurons_noring")
ax.scatter(
    data_df["l_r"][data_df["hdn"]],
    data_df["a_p"][data_df["hdn"]],
    color=COLS["ring"],
    lw=0,
    s=s,
    alpha=0.1,
    rasterized=True,
)
pltltr.savefig("horview_allneurons")

Then, all projections:

In [ ]:
f, axs = plt.subplots(1, 3, figsize=(9, 3))
sel = data_df["hdn"]
planes = "frontal", "horizontal", "sagittal"
ax_labs = "inf.  sup.", "pos.  ant.", "inf.  sup."
locs = [[-20, -150], [-20, -200], [-52, -156]]
s_lab = 20
for i, coords in enumerate(
    [
        [data_df["l_r"], data_df["s_i"]],
        [data_df["l_r"], data_df["a_p"]],
        [data_df["a_p"], data_df["s_i"]],
    ]
):
    axs[i].scatter(
        coords[0],
        -coords[1],
        lw=0,
        s=s,
        alpha=0.03,
        color=(0.3,) * 3,
        label="__nolegend__",
        rasterized=True,
    )
    axs[i].scatter(
        [],
        [],
        lw=0,
        s=s_lab,
        color=(0.3,) * 3,
        label="ROIs",
    )
    axs[i].scatter(
        [],
        [],
        color=COLS["ring"],
        lw=0,
        s=s_lab,
        label="Ring ROIs",
    )
    axs[i].scatter(
        coords[0][sel],
        -coords[1][sel],
        color=COLS["ring"],
        lw=0,
        s=s,
        alpha=0.05,
        label="__nolegend__",
        rasterized=True,
    )

    axs[i].axis("equal")
    axs[i].axis("off")
    axs[i].set_title(planes[i].capitalize() + " view")

    # pltltr.add_anatomy_scalebar(axs[i], plane=planes[i], pos=locs[i], length=30)
    xpos, ypos = locs[i]
    pltltr.add_scalebar(
        axs[i],
        xpos=xpos,
        ypos=ypos,
        xlen=50,
        ylen=50,
        xlabel="50 μm",
        ylabel=ax_labs[i],
        text_spacing_coef=0.15,
    )

axs[2].legend(loc=2, bbox_to_anchor=(0.6, 0.35))
pltltr.savefig("allviews_allneurons", folder="S1")

## MPIN projections

Plot projections over the MPIN atlas and save relative coordinates.

In [ ]:
Z_SCATTER_FACTOR = 0  # 10  # add some random dispersion in z to make all rois visible
data_df = []
for path in tqdm(dataset_folders):
    exp = LotrExperiment(path)
    cent_coords = exp.ipnref_coords_um
    data_dict = dict()
    for i, lab in enumerate(["a_p", "s_i", "l_r"]):
        data_dict.update({lab: cent_coords[:, i]})
    data_dict["s_i"] = (
        data_dict["s_i"] + np.random.rand(len(data_dict["a_p"])) * Z_SCATTER_FACTOR
    )
    data_dict["fid"] = exp.exp_code  # path.name
    data_dict["hdn"] = ~exp.nonhdn_indexes
    data_dict["rpc_angle"] = np.full(exp.n_rois, np.nan)
    data_dict["rpc_angle"][exp.hdn_indexes] = exp.rpc_angles

    data_df.append(pd.DataFrame(data_dict))

data_df = pd.concat(data_df, ignore_index=1)

In [ ]:
atlas = BrainGlobeAtlas("mpin_zfish_1um")
atlas.annotation
# Annoying fixes required to deal with the Baier annotation mess. Will be cleaned in next version of the atlas.
atlas._annotation[[0, -1], :, :] = 0
atlas._annotation[:, [0, -1], :] = 0
atlas._annotation[:, :, [0, -1]] = 0
atlas._annotation[atlas._annotation == 0] = 15000
atlas.structures["root"]["id"] = 1

#  Specify axes limits over all dimensions:
bs = dict(
    frontal=(-10, atlas.shape[2] + 10),
    vertical=(-10, atlas.shape[1] + 10),
    sagittal=(-10, atlas.shape[0] + 10),
)

plotter = pltltr.AtlasPlotter(
    atlas=atlas,
    structures=["root", "hindbrain", "retina", "interpeduncular nucleus"],
    bounds_dict=dict(
        frontal=[bs["vertical"], bs["frontal"]],
        horizontal=[bs["sagittal"], bs["frontal"]],
        sagittal=[bs["vertical"], bs["sagittal"]],
    ),
    mask_slices=dict(
        frontal=slice(460, 470), horizontal=slice(225, 235), sagittal=slice(270, 280)
    ),
    smooth_wnd=50,
)

In [ ]:
ipn_c = data_df[["a_p", "s_i", "l_r"]].values
mpin_c = transform_points(ipn_c, ipn_to_mpin)

In [ ]:
f, axs = plotter.generate_projection_plots(title=True, labels=True)
plotter.axs_scatterplot(axs, mpin_c, alpha=0.08, s=2, lw=0, rasterized=True, c=".6")
plotter.axs_scatterplot(
    axs,
    mpin_c[data_df["hdn"], :],
    c=COLS["ring"],
    alpha=0.08,
    s=2,
    lw=0,
    rasterized=True,
)
pltltr.savefig("rois_in_mpin_ref", folder="S1")

## Breakout plots for individual fish

Here we generate the large panel of the supplementary data with the breakout of individual fish. Lot of code is duplicated as the code was moved from another notebook.

In [ ]:
results_list = []

# Loop over all fish, and compute values for data and after shuffling coords itentity:
for path in dataset_folders:
    exp = LotrExperiment(path)
    sequence = np.arange(len(exp.hdn_indexes))

    # center coords (will be replaced with morphing):
    coords = exp.morphed_coords_um[exp.hdn_indexes, 1:][
        sequence, :
    ]  # shuffle if necessary
    coords[:, 1] = coords[:, 1] + 16  # arbitrary centering

    # And, for plotting, computing an "anatomical phase", aka position around the anatomy:
    anatomical_phase = np.angle(-coords[:, 1] + 1j * -coords[:, 0])

    # Stretch angle to avoid undersampling aroung midline
    anatomical_phase = anatomical_angle_remapping(anatomical_phase)

    r = np.mean(np.sqrt(exp.rpc_scores[:, 0] ** 2 + exp.rpc_scores[:, 1] ** 2))
    results_list.append(
        pd.DataFrame(
            dict(
                rpc1_proj=exp.rpc_scores[:, 0] / r,
                rpc2_proj=exp.rpc_scores[:, 1] / r,
                lr_pos=coords[:, 0],
                ap_pos=coords[:, 1],
                rpc_angle=exp.rpc_angles,
                anatomical_phase=anatomical_phase,
                exp_id=path.name,
                # group=group,
            )
        )
    )

pooled_df = pd.concat(results_list, axis=0).reset_index()

In [ ]:
rhos = []

for sel in pooled_df["exp_id"].unique():
    sel_data = pooled_df[pooled_df["exp_id"] == sel]
    rhos.append(
        -circular_corr(
            sel_data["anatomical_phase"].values, sel_data["rpc_angle"].values
        )
    )
order = np.argsort(rhos)

In [ ]:
exp_codes = []
for path in dataset_folders:
    exp_codes.append(LotrExperiment(path).exp_code)

In [ ]:
p_per_ax = 7
n_cols = 2
n_rows = (len(rhos) + 2) // n_cols

f, all_axs = plt.subplots(
    n_rows + 1,
    p_per_ax * n_cols,
    figsize=(9, 12),
    gridspec_kw=dict(
        left=0.1,
        right=0.99,
        bottom=0.01,
        top=0.95,
        wspace=0.1,
        height_ratios=[
            1,
        ]
        * (n_rows + 1),
        width_ratios=([0.8, 0.8, 0.2, 0.8, 0.2, 0.8, 0.15]) * n_cols,
    ),
)
s = 10
ap = 50
lr = 70

col = ""

axs_flat = all_axs.flatten()

for j in range(2):
    all_axs[0, 3 + j * 2].set(title=["Frontal axis", "Sagittal axis"][j])
    all_axs[0, 3 + j * 2].set_xlabel("rPC angle", fontsize=6)
    all_axs[0, 3 + j * 2].set_ylabel(
        ["Right-left pos. (μm)", "Post.-ant. pos. (μm)"][j], fontsize=6
    )
    all_axs[0, 3 + j * 2].xaxis.set_tick_params(labelsize=6)
    all_axs[0, 3 + j * 2].yaxis.set_tick_params(labelsize=6)

for n in range(len(order) + 1):
    if n == len(order):
        sel_data = pooled_df
        axs = axs_flat[:p_per_ax]
        alpha = 0.1
        exp_code = "all fish"

    else:
        sel = pooled_df["exp_id"].unique()[order[n]]
        sel_data = pooled_df[(pooled_df["exp_id"] == sel)]

        i_plot = (n + 3) * p_per_ax
        axs = axs_flat[i_plot : i_plot + p_per_ax]
        alpha = 1
        exp_code = exp_codes[order[n]]

    axs[0].scatter(
        -sel_data["rpc2_proj"],
        sel_data["rpc1_proj"],
        lw=0,
        c=sel_data["rpc_angle"],
        cmap=COLS["phase"],
        s=s,
        alpha=alpha,
        rasterized=True,
    )

    axs[1].scatter(
        sel_data["lr_pos"],
        sel_data["ap_pos"],
        lw=0,
        c=sel_data["rpc_angle"],
        cmap=COLS["phase"],
        s=s,
        alpha=alpha,
        rasterized=True,
    )
    sc = axs[1].scatter([], [], c=[], cmap=COLS["phase"], vmin=-np.pi, vmax=np.pi)

    rho = circular_corr(
        sel_data["anatomical_phase"].values, sel_data["rpc_angle"].values
    )

    axs[0].text(
        -1.55,
        0,
        f"{exp_code}\nr: {rho:.2f}",
        va="center",
        ha="center",
        rotation=90,
        fontsize=6,
    )
    [axs[j].axis("equal") for j in range(2)]

    axs[1].scatter([-lr, lr, -lr, lr], [ap, ap, -ap, -ap], s=0)
    pltltr.despine(axs[1], "all")
    pltltr.despine(axs[0], "all")

    for j, ax in enumerate(["lr_pos", "ap_pos"]):
        residuals = []

        axs[3 + j * 2].scatter(
            sel_data["rpc_angle"],
            sel_data[ax],
            color="C0",
            s=5,
            lw=0,
            alpha=alpha,
            label="_nolegend_",
            rasterized=True,
        )
        axs[3 + j * 2].set(
            xticks=(-np.pi / 2, 0, np.pi / 2),
            xlim=(-np.pi - 0.2, np.pi + 0.2),
            ylim=[(-60, 60), (-30, 20)][j],
            alpha=alpha,
        )
        if n != len(order):
            axs[3 + j * 2].set(yticklabels=[], xticklabels=[])
        pltltr.despine(axs[3 + j * 2])

    axs[2].axis("off")
    axs[4].axis("off")
    axs[6].axis("off")

[ax.axis("off") for ax in axs_flat[p_per_ax : p_per_ax * 3]]
pltltr.add_anatomy_scalebar(all_axs[0, 1], pos=(-50, -50), length=30, fontsize=6)

pltltr.add_cbar(
    sc,
    all_axs[0, 1],
    inset_loc=(-0.1, -0.2, 0.3, 0.08),
    ticks=[-np.pi + 0.01, 0, np.pi - 0.01],
    ticklabels=["-π", "0", "π"],
    title="rPC angle",
    orientation="horizontal",
    titlesize=6,
    labelsize=6,
)
pltltr.savefig("anatomy_all_fish", folder="S3b")